#### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.



In [1]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

In [2]:
person=Person(name="Alice", age=30, city="New York")
print(person)   


Person(name='Alice', age=30, city='New York')


In [3]:
person1=Person(name="Alice", age=30, city=35)
print(person1)

Person(name='Alice', age=30, city=35)


Here it is not validating the input type checking not occuring

In [27]:
from pydantic import BaseModel
from typing import Optional
from typing import List
from pydantic import Field

In [4]:
from pydantic import BaseModel

In [5]:
class Person1(BaseModel):
    name: str
    age: int
    city: str   
person=Person1(name="Alice", age=30, city="New York")
print(person)


name='Alice' age=30 city='New York'


In [6]:
person1=Person1(name="Alice", age=30, city=35)
# This will raise a validation error because city should be a string
print(person)

ValidationError: 1 validation error for Person1
city
  Input should be a valid string [type=string_type, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [7]:
person2= Person1(name="Alice", age=30, city="35")
print(person2)
# This will not raise an error because city is now a string

name='Alice' age=30 city='35'


#### 2. Model with Optional Fields
Add optional fields using Python's Optional type:



Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

In [8]:
from typing import Optional

In [9]:
class Person2(BaseModel):
    name: str
    age: int
    city: Optional[str] = None  # city is now optional
    department:str
    salary: Optional[float]=None #Optional with default value
    is_active: Optional[bool]=True #optional field with default value

In [10]:
person=Person2(name="Alice", age=30, city="New York", department="Engineering")
print(person)

name='Alice' age=30 city='New York' department='Engineering' salary=None is_active=True


In [11]:
person1=Person2(name="Alice", age=30, city=35, department="Engineering")
# This will raise a validation error because city should be a string
print(person1)

ValidationError: 1 validation error for Person2
city
  Input should be a valid string [type=string_type, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [12]:
person2=Person2(name="Alice", age=30, city="35", department="Engineering",is_active=35)

ValidationError: 1 validation error for Person2
is_active
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=35, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/bool_parsing

In [13]:
person3=Person2(name="Alice", age=30, city="35", department="Engineering",is_active=True,salary="350000")
# This will notraise a validation error because salary converted into  float automatically
print(person3)

name='Alice' age=30 city='35' department='Engineering' salary=350000.0 is_active=True


In [14]:
from typing import List

class Classroom(BaseModel):
    room_number:str
    students: List[str] #List of strings
    capacity:int

In [15]:
# Create a classroom
classroom = Classroom(
    room_number="A101",
    students=("Alice", "Bob", "Charlie"),
    capacity=30
)
print(classroom)

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [16]:
# Create a classroom
classroom1 = Classroom(
    room_number="A101",
    students=("Alice", 123, "Charlie"),
    capacity=30
)
print(classroom1)

ValidationError: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [17]:
try:
    invalid_val=Classroom(room_number="A1",students=["Krish",123],capacity=30)

except Exception as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### 4. Model with Nested Models
Create complex structures with nested models:

In [18]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address  ## Nested Model

In [19]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":"Boston","zip_code":"02108"})

print(customer)

customer_id=1 name='Krish' address=Address(street='Main street', city='Boston', zip_code='02108')


In [20]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":123,"zip_code":"02108"})

print(customer)

ValidationError: 1 validation error for Customer
address.city
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

#### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [21]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int=Field(ge=0)

item=Item(name="Book", price=100000,quantity=10)
print(item)

ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 10000 [type=less_than_equal, input_value=100000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [23]:
class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")


# Examples
user1 = User(username="alice")
print(user1)




username='alice' age=18 email='user@example.com'


default_factory=lambda: "user@example.com"

Instead of directly setting a default value (default="user@example.com"), it uses a default factory, which is a callable (function) that returns the default value when the field is not explicitly set.

lambda: "user@example.com" is an anonymous function that returns "user@example.com".

Why use default_factory?

Useful when the default value is dynamically generated or when you want to ensure a fresh instance is created (e.g., lists, dicts, timestamps).

Even here, though a static string is used, it's showing how you could dynamically generate defaults.

In [24]:
user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)

username='bob' age=25 email='bob@domain.com'


In [25]:
User.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}